    
    Analyses of ribosome profiling data related to the TrmD degron project
    
    Copyright (C) 2021  Allen Buskirk, buskirk@jhmi.edu
    
    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.
    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.
    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [44]:
%load_ext autoreload
%autoreload 2
import os
import ribo_utilities
import ribo_analysis
from BCBio import GFF

FP_names = [
    'trmD_control_FP1',
    'trmD_degron_FP1',
    'trmD_control_FP2',
    'trmD_degron_FP2',
    'trmD_WT_FP',
    'trmD_KO_FP',
    'trmD_control_RNAseq1',
    'trmD_degron_RNAseq1',
    'trmD_control_RNAseq2',
    'trmD_degron_RNAseq2'
    ]
    
TruSeq_names = [
    'trmD_WT_RNAseq',
    'trmD_KO_RNAseq'
    ]

inpath      = '/home/allen/data/trmD/'    
path_script = '/home/allen/code_trmD/'

paths_in = {
    'path_fastq'     : inpath  + 'fastq/', 
    'path_gff'       : path_script + 'coli2.gff',
    'path_bowtie'    : 'bowtie',
    'index_ladder'   : path_script + 'indexes/ladder/ladder',
    'index_trna'     : path_script + 'indexes/MG1655/MG1655_tRNA',
    'index_rrna'     : path_script + 'indexes/MG1655/MG1655_rRNA',
    'index_chr'      : path_script + 'indexes/MG1655/MG1655_genome',
    }

paths_out = {
    'path_filter'       : inpath  + 'filterdata/',
    'path_ladder'       : inpath  + 'alignments/ladder/',
    'path_trna'         : inpath  + 'alignments/tRNA/',
    'path_rrna'         : inpath  + 'alignments/rRNA/',
    'path_chr'          : inpath  + 'alignments/chr/',
    'path_temp'         : inpath  + 'tmpds/',
    'path_density'      : inpath  + 'density/',
    'path_wig'          : inpath  + 'wigfiles/',
    'path_log'          : inpath  + 'log/',
    'path_genelists'    : inpath  + 'analysis/genelists/',
    'path_pauses'       : inpath  + 'analysis/pauses/'
    }

for i in paths_out:
    if not os.path.exists(paths_out[i]):
        os.makedirs(paths_out[i])


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
## use skewer version 0.2.2 to remove the linker from the reads in the FASTQ files 
## for the footprints libraries and the RNAseq from the trmD-degron samples
## which were made from 10-40 nt mRNA fragments

import os

minlength = 10
maxlength = 40
phred_cutoff = 10

linker = 'CTGTAGGCACCATCAATAGATCGGAAGAGCACACGTCTGAACTCCAGTCA'

files = FP_names

print "log saved at paths_out['path_filter']"

for fname in files:
    file_in = paths_in['path_fastq'] + fname + '.fastq'
    file_out = paths_out['path_filter'] + fname

    skewer_command = 'skewer -x %s -Q %d  -l %d -L %d -o %s --quiet %s' % (
        linker, 
        phred_cutoff, 
        minlength,
        maxlength, 
        file_out, 
        file_in 
        )

    os.system(skewer_command)
    print fname + " completed"


log saved at paths_out['path_filter']
trmD_control_FP1 completed
trmD_degron_FP1 completed


In [5]:
## for TruSeq, use skewer for quality filtering only, not trimming the linker

import os

phred_cutoff = 10

files = TruSeq_names

print "log saved at paths_out['path_filter']"

for fname in files:
    file_in = paths_in['path_fastq'] + fname + '.fastq'
    file_out = paths_out['path_filter'] + fname

    skewer_command = 'skewer -Q %d -o %s --quiet %s' % (
        phred_cutoff, 
        file_out, 
        file_in 
        )

    os.system(skewer_command)
    print fname + " completed"


log saved at paths_out['path_filter']
trmD_WT_RNAseq completed
trmD_KO_RNAseq completed


In [6]:
# alignments using bowtie 1.2.2
# this function aligns to a ladder index, then the tRNA index, then the rRNA index, then the chromosome
# tRNA and rRNA match alignments stored in the tmpds folder
# makes tmpds folder and alignment folders automatically

import os
import sys

# files = FP_names + TruSeq_names
files = TruSeq_names

######################################################

path_l = paths_out['path_ladder']
path_t = paths_out['path_trna']
path_r = paths_out['path_rrna']
path_chr = paths_out['path_chr']
path_temp = paths_out['path_temp']

if os.path.exists(paths_out['path_filter']+files[0]+'_filtered_trimmed.fastq'):
    trimmed = '_filtered_trimmed.fastq '
elif os.path.exists(paths_out['path_filter']+files[0]+'-trimmed.fastq'):
    trimmed = '-trimmed.fastq '
else:
    sys.exit("error: trimmed file name")
    
for fname in files:
    print fname
    
    #first, align to ladder index to subtract any ladder oligos that were cloned
    bowtie_l = paths_in['path_bowtie'] + ' -v 2 -y -m 1 -a --best --strata -S -p 2 --un '
    bowtie_l += path_l + fname + '_nomatch.fastq --max ' + path_l + fname + '_multi.fastq --al '
    bowtie_l += path_l + fname + '_match.fastq ' + paths_in['index_ladder'] + ' '
    bowtie_l += paths_out['path_filter'] + fname + trimmed + ' 1> ' + path_temp + fname + '_ladder_match.SAM'
    bowtie_l += ' 2> ' + paths_out['path_log'] + fname
    os.system(bowtie_l)

    # second, align to tRNA index
    bowtie_tRNA = paths_in['path_bowtie'] + ' -v 2 -y -m 1 -a --best --strata -S -p 2 --un '
    bowtie_tRNA += path_t + fname + '_nomatch.fastq --max ' + path_t + fname + '_multi.fastq --al '
    bowtie_tRNA += path_t + fname + '_match.fastq ' + paths_in['index_trna'] + ' '
    bowtie_tRNA += path_l + fname + '_nomatch.fastq ' + ' 1> ' + path_temp + fname + '_tRNA_match.SAM'
    bowtie_tRNA += ' 2>> ' + paths_out['path_log'] + fname 
    os.system(bowtie_tRNA)

    # third, align to the rRNA index
    bowtie_rRNA = paths_in['path_bowtie'] + ' -v 2 -y -m 1 -a --best --strata -S -p 2 --un '
    bowtie_rRNA += path_r + fname + '_nomatch.fastq --max ' + path_r + fname + '_multi.fastq --al '
    bowtie_rRNA += path_r + fname + '_match.fastq ' + paths_in['index_rrna'] + ' '
    bowtie_rRNA += path_t + fname + '_nomatch.fastq ' + ' 1> ' + path_temp + fname + '_rRNA_match.SAM'
    bowtie_rRNA += ' 2>> ' + paths_out['path_log'] + fname 
    os.system(bowtie_rRNA)

    # finally align to the chr index
    bowtie_chr = paths_in['path_bowtie'] + ' -v 2 -y -m 1 -a --best --strata -S -p 2 --un '
    bowtie_chr += path_chr + fname + '_nomatch.fastq --max ' + path_chr + fname + '_multi.fastq --al '
    bowtie_chr += path_chr + fname + '_match.fastq ' + paths_in['index_chr'] + ' '
    bowtie_chr += path_r + fname + '_nomatch.fastq ' + ' 1> ' + path_chr + fname + '_match.SAM'
    bowtie_chr += ' 2>> ' + paths_out['path_log'] + fname 
    os.system(bowtie_chr)

trmD_WT_RNAseq
trmD_KO_RNAseq


In [4]:
### creating ribosome density files, saving pickle file and WIG file

files = FP_names
minlength = 10
maxlength = 40
threads = 4

ribo_utilities.run_density(files, minlength, maxlength, threads, paths_in, paths_out, 'ribo_seq')


-----DENSITY-----

Files to condense: trmD_control_FP1, trmD_degron_FP1, trmD_control_FP2, trmD_degron_FP2, trmD_WT_FP, trmD_KO_FP, trmD_control_RNAseq1, trmD_degron_RNAseq1, trmD_control_RNAseq2, trmD_degron_RNAseq2

	Started density at 2021-02-04 16:01:28.536678
	trmD_degron_RNAseq2 3222468
	trmD_control_RNAseq2 4138978
	trmD_degron_RNAseq1 4061634
	trmD_KO_FP 7137690
	trmD_WT_FP 8648301
	trmD_control_RNAseq1 8449124
	trmD_degron_FP1 19918072
	trmD_degron_FP2 19998980
	trmD_control_FP1 20466441
	trmD_control_FP2 20760261
	Finished density at 2021-02-04 16:02:35.301536
	COMPLETED DENSITY


In [31]:
### for TruSeq creating the same density files, pickle and WIG
### adding "TruSeq" to the argument flips the strands
###
### by hand, you must collect all the output and save in the reads.csv file in the density folder with format:
### filename,mapped_reads

files = TruSeq_names
minlength = 40
maxlength = 60
threads = 4

ribo_utilities.run_density(files, minlength, maxlength, threads, paths_in, paths_out, 'TruSeq')


-----DENSITY-----

Files to condense: trmD_WT_RNAseq, trmD_KO_RNAseq

	Started density at 2021-02-05 15:25:44.922732
	trmD_WT_RNAseq 19239048
	trmD_KO_RNAseq 32082308
	Finished density at 2021-02-05 15:27:25.353326
	COMPLETED DENSITY


In [33]:
### generate a list of genes with the rpkm values and reads to quantitate expression

files = TruSeq_names

thresh = -1      # threshold value in reads per codon, -1 gives you all genes
shift = 0        # no shift for TruSeq samples

# the totalreads for each library is stored in reads.csv
# and is needed to calculate reads per codon, since the density
# files are stored as rpm not absolute counts
reads_file = paths_out['path_density'] + 'reads.csv'
f_reads = open(reads_file, 'rU')
readlist = ribo_analysis.readindict(f_reads)
f_reads.close()

GFFgen = GFF.parse(paths_in['path_gff'])
chrom = GFFgen.next()

for fname in files:
    print ""
    print fname
    
    totalreads = int(readlist[fname][0])

    counts0 = ribo_utilities.unPickle(paths_out['path_density'] + fname + '_plus')
    counts1 = ribo_utilities.unPickle(paths_out['path_density'] + fname + '_minus')
    counts = [counts0[chrom.id],counts1[chrom.id]]
    
    mgl = ribo_analysis.makegenelist(counts, chrom, shift, thresh, totalreads)
    ribo_analysis.writedicttoexcel(mgl, paths_out['path_genelists'] + fname + "_genelist")


trmD_WT_RNAseq
Genes below threshold = 0
Genes dropped by givegene (overlap, undesirable features, etc.) = 872
Genes in list = 4146

trmD_KO_RNAseq
Genes below threshold = 0
Genes dropped by givegene (overlap, undesirable features, etc.) = 872
Genes in list = 4146


In [13]:
files = FP_names

thresh = -1      # threshold value in reads per codon, -1 gives you all genes
shift = -15      # shifting reads to line up with ribosome sites, -15 gives you the P site in E coli

# the totalreads for each library is stored in reads.csv
# and is needed to calculate reads per codon, since the density
# files are stored as rpm not absolute counts
reads_file = paths_out['path_density'] + 'reads.csv'
f_reads = open(reads_file, 'rU')
readlist = ribo_analysis.readindict(f_reads)
f_reads.close()

GFFgen = GFF.parse(paths_in['path_gff'])
chrom = GFFgen.next()

for fname in files:
    print ""
    print fname
    
    totalreads = int(readlist[fname][0])

    counts0 = ribo_utilities.unPickle(paths_out['path_density'] + fname + '_plus')
    counts1 = ribo_utilities.unPickle(paths_out['path_density'] + fname + '_minus')
    counts = [counts0[chrom.id],counts1[chrom.id]]
    
    mgl = ribo_analysis.makegenelist(counts, chrom, shift, thresh, totalreads)
    ribo_analysis.writedicttoexcel(mgl, paths_out['path_genelists'] + fname + "_genelist")

In [41]:
### code for calculating ribosome pausing at the A site codon for all 20 amino acids 
### the pause score is called "oldpause" and is found in the csv file
### the binary file output contains 21 plots (end-to-end) visualized with Igor Pro in the figures
### plots in the binary file occur in the same order as the csv file

files = ['trmD_control_FP1',
    'trmD_degron_FP1',
    'trmD_control_FP2',
    'trmD_degron_FP2',
        ]
    
motifsize = 1           # Size of the motif, in units of bp or aa as appropriate
scorewin = [-3,-2]      # For pause scores, the region to include as density in numerator
                        #       Scorewin[0] = 0 is the start of motif
                        #       e.g. [0,3] gives 3 nt density: positions 0, 1, and 2
                        #       use -3,-2 for the A site codon 
avgwin = [50,50]        # For the size of the window. [0] is nt upstream, [1] downstream
                        #   of the start of the motif
inframe = 2             # 0 = no frame, 1 = in frame, 2 = amino acids
thresh = 0.1            # cutoff for inclusion for genes; gene average must be above thresh reads per codon
shift = -15             # shifts density to line up with the ribosomal P site codon 
threads = 4

# the totalreads for each library is stored in reads.csv and is needed to calculate reads per codon
# since the density files are stored as rpm not absolute reads
reads_file = paths_out['path_density'] + 'reads.csv'
f_reads = open(reads_file, 'rU')
readlist = ribo_analysis.readindict(f_reads)
f_reads.close()

GFFgen = GFF.parse(paths_in['path_gff'])
chrom = GFFgen.next()

arguments = []

for fname in files:
    print fname
    
    totalreads = int(readlist[fname][0])

    counts0 = ribo_utilities.unPickle(paths_out['path_density'] + fname + '_plus')
    counts1 = ribo_utilities.unPickle(paths_out['path_density'] + fname + '_minus')
    counts = [counts0[chrom.id],counts1[chrom.id]]
        
    outfilestring = paths_out['path_pauses'] + fname + '_A_site_aa'
    
    arg = [chrom, counts, outfilestring, motifsize, inframe, thresh, totalreads, shift, avgwin, scorewin]
    arguments.append(arg)

ribo_utilities.multiprocess(ribo_analysis.motifavg_ab_wf, arguments, threads)


trmD_control_FP1
trmD_degron_FP1
trmD_control_FP2
trmD_degron_FP2
trmD_WT_FP
trmD_KO_FP


In [43]:
### some of the short reads in the two Trm5 samples don't really have a 3'-end 
### corresponding to the 3'-boundary of the ribosome, so remove those for pausing analyses
### focusing only on reads 24 - 40 nt in length
### this will create density and wig files named _long for these analyses

files = ['trmD_WT_FP', 'trmD_KO_FP']
minlength = 24
maxlength = 40
threads = 4

ribo_utilities.run_density(files, minlength, maxlength, threads, paths_in, paths_out, 'ribo_seq')


-----DENSITY-----

Files to condense: trmD_WT_FP, trmD_KO_FP

	Started density at 2021-02-08 10:25:02.911870
	trmD_KO_FP 4158284
	trmD_WT_FP 5119259
	Finished density at 2021-02-08 10:25:42.247537
	COMPLETED DENSITY


In [48]:
### pausing analyses for the long reads of the trmD_KO and _WT strains
### code for calculating ribosome pausing at the A site codon for all 20 amino acids 
### scorewin is slightly different here because of the way the reads line up 
### (these were made with a different lysis buffer)

files = ['trmD_WT_FP_long', 'trmD_KO_FP_long']
    
motifsize = 1           # Size of the motif, in units of bp or aa as appropriate
scorewin = [-4,-2]      # For pause scores, the region to include as density in numerator
                        #       Scorewin[0] = 0 is the start of motif
                        #       e.g. [0,3] gives 3 nt density: positions 0, 1, and 2
                        #       use -3,-2 for the A site codon 
avgwin = [50,50]        # For the size of the window. [0] is nt upstream, [1] downstream
                        #   of the start of the motif
inframe = 2             # 0 = no frame, 1 = in frame, 2 = amino acids
thresh = 0.1            # cutoff for inclusion for genes; gene average must be above thresh reads per codon
shift = -15             # shifts density to line up with the ribosomal P site codon 
threads = 4

# the totalreads for each library is stored in reads.csv and is needed to calculate reads per codon
# since the density files are stored as rpm not absolute reads
reads_file = paths_out['path_density'] + 'reads.csv'
f_reads = open(reads_file, 'rU')
readlist = ribo_analysis.readindict(f_reads)
f_reads.close()

GFFgen = GFF.parse(paths_in['path_gff'])
chrom = GFFgen.next()

arguments = []

for fname in files:
    print fname
    
    totalreads = int(readlist[fname][0])

    counts0 = ribo_utilities.unPickle(paths_out['path_density'] + fname + '_plus')
    counts1 = ribo_utilities.unPickle(paths_out['path_density'] + fname + '_minus')
    counts = [counts0[chrom.id],counts1[chrom.id]]
        
    outfilestring = paths_out['path_pauses'] + fname + '_A_site_aa'
    
    arg = [chrom, counts, outfilestring, motifsize, inframe, thresh, totalreads, shift, avgwin, scorewin]
    arguments.append(arg)

ribo_utilities.multiprocess(ribo_analysis.motifavg_ab_wf, arguments, threads)


trmD_WT_FP_long
trmD_KO_FP_long


In [46]:
### code for calculating ribosome pausing at the A site codon for all 64 codons
### the pause score is called "oldpause" and is found in the csv file
### the binary file output contains 63 plots (end-to-end) visualized with Igor Pro in the figures

files = ['trmD_control_FP1',
    'trmD_degron_FP1',
    'trmD_control_FP2',
    'trmD_degron_FP2',
        ]
    
motifsize = 3           # Size of the motif, in units of bp or aa as appropriate
scorewin = [-3,-2]      # For pause scores, the region to include as density in numerator
                        #       Scorewin[0] = 0 is the start of motif
                        #       e.g. [0,3] gives 3 nt density: positions 0, 1, and 2
                        #       use -3,-2 for the A site codon 
avgwin = [50,50]        # For the size of the window. [0] is nt upstream, [1] downstream
                        #   of the start of the motif
inframe = 1             # 0 = no frame, 1 = in frame, 2 = amino acids
thresh = 0.1            # cutoff for inclusion for genes; gene average must be above thresh reads per codon
shift = -15             # shifts density to line up with the ribosomal P site codon 
threads = 4

# the totalreads for each library is stored in reads.csv and is needed to calculate reads per codon
# since the density files are stored as rpm not absolute reads
reads_file = paths_out['path_density'] + 'reads.csv'
f_reads = open(reads_file, 'rU')
readlist = ribo_analysis.readindict(f_reads)
f_reads.close()

GFFgen = GFF.parse(paths_in['path_gff'])
chrom = GFFgen.next()

arguments = []

for fname in files:
    print fname
    
    totalreads = int(readlist[fname][0])

    counts0 = ribo_utilities.unPickle(paths_out['path_density'] + fname + '_plus')
    counts1 = ribo_utilities.unPickle(paths_out['path_density'] + fname + '_minus')
    counts = [counts0[chrom.id],counts1[chrom.id]]
        
    outfilestring = paths_out['path_pauses'] + fname + '_A_site_codons'
    
    arg = [chrom, counts, outfilestring, motifsize, inframe, thresh, totalreads, shift, avgwin, scorewin]
    arguments.append(arg)

ribo_utilities.multiprocess(ribo_analysis.motifavg_ab_wf, arguments, threads)


trmD_control_FP1
trmD_degron_FP1
trmD_control_FP2
trmD_degron_FP2


In [49]:
### pausing analyses for the long reads of the trmD_KO and _WT strains
### code for calculating ribosome pausing at the A site codon for all 64 codons 
### scorewin is slightly different here because of the way the reads line up 
### (these were made with a different lysis buffer)

files = ['trmD_WT_FP_long', 'trmD_KO_FP_long']
    
motifsize = 3           # Size of the motif, in units of bp or aa as appropriate
scorewin = [-4,-2]      # For pause scores, the region to include as density in numerator
                        #       Scorewin[0] = 0 is the start of motif
                        #       e.g. [0,3] gives 3 nt density: positions 0, 1, and 2
                        #       use -3,-2 for the A site codon 
avgwin = [50,50]        # For the size of the window. [0] is nt upstream, [1] downstream
                        #   of the start of the motif
inframe = 1             # 0 = no frame, 1 = in frame, 2 = amino acids
thresh = 0.1            # cutoff for inclusion for genes; gene average must be above thresh reads per codon
shift = -15             # shifts density to line up with the ribosomal P site codon 
threads = 4

# the totalreads for each library is stored in reads.csv and is needed to calculate reads per codon
# since the density files are stored as rpm not absolute reads
reads_file = paths_out['path_density'] + 'reads.csv'
f_reads = open(reads_file, 'rU')
readlist = ribo_analysis.readindict(f_reads)
f_reads.close()

GFFgen = GFF.parse(paths_in['path_gff'])
chrom = GFFgen.next()

arguments = []

for fname in files:
    print fname
    
    totalreads = int(readlist[fname][0])

    counts0 = ribo_utilities.unPickle(paths_out['path_density'] + fname + '_plus')
    counts1 = ribo_utilities.unPickle(paths_out['path_density'] + fname + '_minus')
    counts = [counts0[chrom.id],counts1[chrom.id]]
        
    outfilestring = paths_out['path_pauses'] + fname + '_A_site_codons'
    
    arg = [chrom, counts, outfilestring, motifsize, inframe, thresh, totalreads, shift, avgwin, scorewin]
    arguments.append(arg)

ribo_utilities.multiprocess(ribo_analysis.motifavg_ab_wf, arguments, threads)


trmD_WT_FP_long
trmD_KO_FP_long
